## Force a language model not to use the letter 'e'

![](./assets/Plaque_perec.jpg){width=60% fig-align="center"}

### Introduction

Georges Perec (1936-1982) was a French novelist, famous among other things for writing a $300$-page novel called « La disparition  », without using during the whole novel the letter 'e' (the most used vowel in French). The English translation "A Void" also kept that constraint, while the Spanish translation "El Secuestro" is written without using the letter 'a' (the most used vowel in Spanish). 

This type of writing is called a *lipogram* and it is a type of constrained writing. Writing a lipogram is a trivial task when avoiding uncommon letters (for example, z, j, q, or x), but it is much more challenging to avoid common letters (for example, e, t, or a) since we must omit many ordinary words ^[https://en.wikipedia.org/wiki/Lipogram].

In this post, we want to force a language model not to use the letter 'e' in its responses.

### The Challenge

Let's first see that this is a difficult task for a language model.

First, let's download a small language model and its tokenizer.

In [1]:
#| echo: false
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
#| code-fold: true
#| code-summary: "Show the code"
import torch
from typing import List
from threading import Thread
from unicodedata import normalize
from transformers.generation import LogitsProcessor
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from transformers.generation import LogitsProcessor
from IPython.display import Markdown

# Auto select device (CUDA > MPS > CPU)
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
model_id = "Qwen/Qwen3-0.6B"
model = AutoModelForCausalLM.from_pretrained(
    model_id, cache_dir="/big_storage/llms/hf_models/"
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)

Let's ask the following:

> Tell me a short story without using the letter 'e'

Here is the language model's response:

In [3]:
#| code-fold: true
#| code-summary: "Show the code"

user_input = "Tell me a short story without using the letter 'e'"

def generate_response(user_input, logits_processor=[], enable_thinking=False):
    messages = [
        {"role": "user", "content": user_input},
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=enable_thinking,
    )

    model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    prompt_length = model_inputs['input_ids'].shape[-1]

    generation_kwargs = dict(
        model_inputs,
        streamer=streamer,
        logits_processor=logits_processor,
        max_new_tokens=4 * 1024,
        do_sample=False,
        temperature=1.0,
        top_p=1.0,
        top_k=50,
    )

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    assistant_response = ""
    for chunk in streamer:
        assistant_response += chunk
        # print(chunk, end="")

    clean_assistant_response = assistant_response.split("<|im_end|>")[0]

    thread.join()
    return clean_assistant_response

from IPython.display import display, Markdown

def callout(text, kind="info"):
    colors = {
        "info": ("#D1ECF1", "#0C5460"),
        "warning": ("#FFF3CD", "#856404"),
        "success": ("#D4EDDA", "#155724"),
        "danger": ("#F8D7DA", "#721C24"),
    }
    bg, fg = colors.get(kind, colors["info"])
    md = f"""
<div style="
    background-color: {bg};
    color: {fg};
    border-left: 4px solid {fg};
    padding: 8px 12px;
    border-radius: 4px;">
{text}
</div>
"""
    display(Markdown(md))

assistant_response = generate_response(user_input)
callout(assistant_response)


<div style="
    background-color: #D1ECF1;
    color: #0C5460;
    border-left: 4px solid #0C5460;
    padding: 8px 12px;
    border-radius: 4px;">
In a quiet village, a young girl named Lily found a hidden book tucked under a tree. She loved reading and spent her days there, surrounded by stories that brought her joy. One day, she discovered a secret message hidden in the book: *“The world is full of wonders, and I am here to share them.”* With a smile, she returned to her village, carrying the message with her.
</div>



A complete failure!!!

Now, this is a small language model, so let's see GPT-4o response: 

![](./assets/lipogram_chatgpt.jpg)

Another failure!!!

Here is the [link to that conversation](https://chatgpt.com/share/68790887-6d9c-800a-a928-ff882e6cb198).

### The Solution

In my [previous post](https://alonsosilvaallende.github.io/blog/posts/2025-07-16-Understanding-Logits-Processors/Understanding_Logits_Processors.html), I discussed about logits processors and how they can be used to force language models to do all sorts of things. Well, we are going to use a logits processor to accomplish this task.

Let's see that 'e' is indeed one of the most used vowels. First, for each vowel we find all the tokens that contain that vowel.

In [4]:
#| code-fold: true
#| code-summary: "Show the code"
list_of_vowels = ["a", "e", "i", "o", "u"]
tokens_per_vowel = dict()
for vowel in list_of_vowels:
    tokens_containing_a_given_vowel = []
    for token_id in range(tokenizer.vocab_size):
        # check uppercase and accents
        if (
            vowel in tokenizer.decode(token_id)
            or vowel.upper() in tokenizer.decode(token_id)
            or normalize('NFC', f"{vowel}\u0300") in tokenizer.decode(token_id)
            or normalize('NFC', f"{vowel}\u0301") in tokenizer.decode(token_id)
            or normalize('NFC', f"{vowel}\u0302") in tokenizer.decode(token_id)
            or normalize('NFC', f"{vowel}\u0303") in tokenizer.decode(token_id)
            or normalize('NFC', f"{vowel}\u0308") in tokenizer.decode(token_id)
        ):
            tokens_containing_a_given_vowel.append(token_id)
    tokens_per_vowel[vowel] = tokens_containing_a_given_vowel

Let's verify the code is working:

In [5]:
#| code-fold: true
#| code-summary: "Show the code"
for vowel in list_of_vowels:
    print(f"Tokens that contain letter {vowel}: {" ".join(tokenizer.convert_ids_to_tokens(tokens_per_vowel[vowel][:15]))}...")

Tokens that contain letter a: A a Ġa at an ar al as am Ġand ad ate ag ay ĠA...
Tokens that contain letter e: E e er re en le Ġthe es ed et el ent Ġre se ex...
Tokens that contain letter i: I i in it is ing ion ic Ġin id im il if ig iv...
Tokens that contain letter o: O o on or ou ion Ġo ro Ġto Ġof om ol od ot ow...
Tokens that contain letter u: U u ou ur ut us un ul ue um ub urn out turn our...


It is working. Let's see how many tokens per vowel are there in this tokenizer:

In [6]:
#| code-fold: true
#| code-summary: "Show the code"
print(f"Total number of vocab tokens: {tokenizer.vocab_size:,}")
for vowel in list_of_vowels:
    print(f"There are {len(tokens_per_vowel[vowel]):,} tokens that contain vowel '{vowel}' or {len(tokens_per_vowel[vowel])/tokenizer.vocab_size:.2%} of tokens")

Total number of vocab tokens: 151,643
There are 36,963 tokens that contain vowel 'a' or 24.38% of tokens
There are 47,861 tokens that contain vowel 'e' or 31.56% of tokens
There are 34,809 tokens that contain vowel 'i' or 22.95% of tokens
There are 30,287 tokens that contain vowel 'o' or 19.97% of tokens
There are 16,789 tokens that contain vowel 'u' or 11.07% of tokens


OK, so the letter 'e' is indeed a quite used vowel.

We want to create a logits processor that will forbid all the tokens that contain the letter 'e'. 

We first create a logits processor that will receive a list of forbidden tokens and that it will lower the raw scores of the forbiden tokens to minus infinite as follows:

In [7]:
#| echo: true
class GeorgePerecLogitsProcessor(LogitsProcessor):
    def __init__(self, forbidden_tokens: List[int]):
        self.forbidden_tokens = forbidden_tokens

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor
    ) -> torch.FloatTensor:
        scores_processed = scores.clone()
        vocab_tensor = torch.arange(scores.shape[-1], device=scores.device)
        forbidden_tokens = torch.tensor(self.forbidden_tokens, device=scores.device)
        forbidden_tokens_mask = torch.isin(vocab_tensor, forbidden_tokens)
        scores_processed = torch.where(forbidden_tokens_mask, -torch.inf, scores)

        return scores_processed

We can instantiate the logits processor by putting all the tokens that contain the letter 'e' as forbidden tokens:

In [8]:
#| echo: true
logits_processors = [
    GeorgePerecLogitsProcessor(
        forbidden_tokens=tokens_per_vowel["e"],
    )
]

We can ask the language model the following:

> Tell me a story about pirates and adventures

Here is the language model's response

In [9]:
#| code-fold: true
#| code-summary: "Show the code"
user_input = "Tell me a story about pirates and adventures"
assistant_response = generate_response(user_input, logits_processor=logits_processors)
callout(assistant_response)


<div style="
    background-color: #D1ECF1;
    color: #0C5460;
    border-left: 4px solid #0C5460;
    padding: 8px 12px;
    border-radius: 4px;">
**A Story of Shadows and Stars**

In a distant land far from civilization, known as *Varkhara*, a small island shrouding in mist and fog, lay a kingdom of sailors and warriors. It was said that this land was born from a storm, and that its only way to find its way was to sail through its own storms.

Long ago, a young man, **Karan**, was born to a family of fish and sailors. His family had always known that to find a path to glory, a man must sail through storms. So, at a young and curious mind, Karan took his first sail. With a small boat and a compass, and a spirit of daring, Karan rowdily rowing his way through a storm.

But as Karan rowdily rowing his way through a storm, a shadow looms on his path. A dark, old man, **Malik**, a long-forging captain, had always known that to sail through storms, a man must not only row but also fight. Malik had always taught Karan that storms could not only bring rain but also bring warriors.

With Malik’s wisdom and Karan’s daring spirit, Karan and Malik soon found a way to sail through storms. With a small boat and a compass, and a spirit of daring, Karan rowdily rowing his way through a storm.

But as Karan rowdily rowing his way through a storm, a shadow looms on his path. A dark, old man, **Malik**, a long-forging captain, had always known that to sail through storms, a man must not only row but also fight. Malik had always taught Karan that storms could not only bring rain but also bring warriors.

With Malik’s wisdom and Karan’s daring spirit, Karan and Malik soon found a way to sail through storms. With a small boat and a compass, and a spirit of daring, Karan rowdily rowing his way through a storm.

And so, Karan and Malik, two sailors from Varkhara, found a way to sail through storms, not just through storms, but through *war*.
</div>


Perfect! 

Here is an app to talk to a language model like this one yourself (it is running on CPU, so it might be slow):

In [12]:
#| echo: false
from IPython.display import IFrame

IFrame("https://alonsosilva-georgesperecassistant.hf.space", width=850, height=600)

Here is the [link to the app](https://alonsosilva-georgesperecassistant.hf.space).

For faster inference, you can also clone [this repo](https://github.com/alonsosilvaallende/Georges-Perec-Assistant) and run it on your machine.

## Conclusions

We successfully forced a small language model (0.6B parameters) not to use the letter 'e'. I think it's super interesting to see applications in which a basic logits processor can reliably make a small language model fulfill a task where much larger models fail.